In [53]:
#Importing dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import numpy as np
import config

In [54]:
#Extracting academy_awards_1927-2015 as a Pandas DataFrame
academy_data_csv = "Resources/academy_awards_1927-2015.csv"
academy_data_df = pd.read_csv(academy_data_csv)

# display raw data
academy_data_df.head()

,Year,Ceremony,Award,Winner,Name,Film
0,1927/1928,1,Actor,NaN,Richard Barthelmess,The Noose
1,1927/1928,1,Actor,1.0,Emil Jannings,The Last Command
2,1927/1928,1,Actress,NaN,Louise Dresser,A Ship Comes In
3,1927/1928,1,Actress,1.0,Janet Gaynor,7th Heaven
4,1927/1928,1,Actress,NaN,Gloria Swanson,Sadie Thompson


In [55]:
#Extracting academy_awards_1927-2015 as a Pandas DataFrame
imdb_top_csv = "Resources/imdb_top_1000.csv"
imdb_top_df = pd.read_csv(imdb_top_csv)

# display raw data
imdb_top_df.head(2)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"


<h1>Transfrom Data</h1>

<H3>IMDB data</H3>

In [56]:
# Combine star columns into 1 combined column of actors
imdb_top_df['actors'] = imdb_top_df[imdb_top_df.columns[10:15]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

In [57]:
# Drop unnecessary columns
dopped_imdb_top_df = imdb_top_df.drop(columns=['Overview', 'Certificate', "Poster_Link", "Star1", "Star2", "Star3", "Star4"])

# Remove the "min" from the runtime column
dopped_imdb_top_df["Runtime"] = dopped_imdb_top_df["Runtime"].str.rstrip(' min')

# Rename column headers
imdb_top_1000_df = dopped_imdb_top_df.rename(columns={"Series_Title": "film_name",
                                                "Released_Year": "released_year",
                                                "Runtime": "runtime",
                                                "Genre": "genre",
                                                "IMDB_Rating": "imdb_rating",
                                                "Meta_score": "meta_score",
                                                "Director": "director",
                                                "actors": "actors",
                                                "No_of_Votes": "votes",
                                                "Gross": "gross"
                                                })

# Drop duplicate film names
imdb_top_1000_df = pd.DataFrame(imdb_top_1000_df[imdb_top_1000_df["film_name"] != "Drishyam"])

In [58]:
# remove commas from gross so it can be conveted to int data type
imdb_top_1000_df["gross"] = imdb_top_1000_df['gross'].replace(to_replace=",", value="", regex=True)

# remove pg from released_year so it can be conveted to int data type
imdb_top_1000_df["released_year"] = imdb_top_1000_df['released_year'].replace(to_replace="PG", value="0", regex=True)

# replaces a NaN value with 0 so we can convert the column to int
imdb_top_1000_df["gross"] = imdb_top_1000_df['gross'].fillna(0)

# convert data types
imdb_top_1000_df["gross"] = imdb_top_1000_df["gross"].astype('int')
imdb_top_1000_df["released_year"] = imdb_top_1000_df["released_year"].astype('int')
imdb_top_1000_df["runtime"] = imdb_top_1000_df["runtime"].astype('int')

In [59]:
# display transfomed data
imdb_top_1000_df.head(5)

,film_name,released_year,runtime,genre,imdb_rating,meta_score,director,votes,gross,actors
0,The Shawshank Redemption,1994,142,Drama,9.3,80.0,Frank Darabont,2343110,28341469,"Tim Robbins,Morgan Freeman,Bob Gunton,William ..."
1,The Godfather,1972,175,"Crime, Drama",9.2,100.0,Francis Ford Coppola,1620367,134966411,"Marlon Brando,Al Pacino,James Caan,Diane Keato..."
2,The Dark Knight,2008,152,"Action, Crime, Drama",9.0,84.0,Christopher Nolan,2303232,534858444,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich..."
3,The Godfather: Part II,1974,202,"Crime, Drama",9.0,90.0,Francis Ford Coppola,1129952,57300000,"Al Pacino,Robert De Niro,Robert Duvall,Diane K..."
4,12 Angry Men,1957,96,"Crime, Drama",9.0,96.0,Sidney Lumet,689845,4360000,"Henry Fonda,Lee J. Cobb,Martin Balsam,John Fie..."


<H3>Academy data</H3>

In [60]:
# Convert winner column to be boolean values
academy_data_df["Winner"] = academy_data_df['Winner'].replace(1, True).fillna(False)

In [61]:
# Drop the first rows that have multiple year values.
academy_data_df = academy_data_df.iloc[266: , :]

# Reset the index
academy_data_df = academy_data_df.reset_index(drop=True)

In [62]:
#------------ The raw data had some categories listed in the wrong columns: winner and film name ---------

# We create a new df with the cateogories that have the data in the correct columns
dropped_academy_data_df_1 = academy_data_df[(academy_data_df.Award == "Actor")\
                | (academy_data_df.Award == "Actor in a Leading Role")\
                | (academy_data_df.Award == "Actor in a Supporting Role")\
                | (academy_data_df.Award == "Actress")\
                | (academy_data_df.Award == "Actress in a Leading Role")\
                | (academy_data_df.Award == "Actress in a Supporting Role")]

# Rename column headers
dropped_academy_data_df_1 = dropped_academy_data_df_1.rename(columns={"Film": "film_name",
                                                "Year": "ceremony_year",
                                                "Award": "award",
                                                "Winner": "winner",
                                                "Name": "winner_name"
                                                })


# Create another df with the categories that have the data in the wrong columns
dropped_academy_data_df_2 = academy_data_df[(academy_data_df.Award == "Animated Feature Film")\
                | (academy_data_df.Award == "Best Motion Picture")\
                | (academy_data_df.Award == "Best Picture")\
                | (academy_data_df.Award == "Directing")\
                | (academy_data_df.Award == "Documentary")\
                | (academy_data_df.Award == "Visual Effects")]

# Rename column headers for dropped df 2 BUT making sure the film name and winnre name columns are swapped
dropped_academy_data_df_2 = dropped_academy_data_df_2.rename(columns={"Name": "film_name",
                                                "Year": "ceremony_year",
                                                "Award": "award",
                                                "Winner": "winner",
                                                "Film": "winner_name"
                                                })

# Now we join the wo df's together using the pandas concat method
clean_academy_data_df = pd.concat([dropped_academy_data_df_1, dropped_academy_data_df_2], axis=0)

In [63]:
# Drop unnecessary columns
clean_academy_data_df = clean_academy_data_df.drop(columns=['Ceremony'])

# convert data type
clean_academy_data_df["ceremony_year"] = clean_academy_data_df["ceremony_year"].astype('int')

In [64]:
# display transformed data
clean_academy_data_df.head(5)

,ceremony_year,award,winner,winner_name,film_name
0,1934,Actor,True,Clark Gable,It Happened One Night
1,1934,Actor,False,Frank Morgan,The Affairs of Cellini
2,1934,Actor,False,William Powell,The Thin Man
3,1934,Actress,True,Claudette Colbert,It Happened One Night
4,1934,Actress,False,Bette Davis,Of Human Bondage


<H1>sqlalchemy</H1>

In [65]:
#Connecting to local database using config for all personal data
rds_connection_string = f'{config.protocol}://{config.username}:{config.password}@{config.host}:{config.port}/{config.database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [66]:
insp.get_table_names()

['imdb_top_1000', 'academy_award_data']

In [67]:
imdb_top_1000_df.to_sql(name='imdb_top_1000', con=engine, if_exists='append', index=False)

In [68]:
clean_academy_data_df.to_sql(name='academy_award_data', con=engine, if_exists='append', index=False)